In [ ]:
from flask import Flask, render_template, request, url_for
import sys, ast, os, json
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
import seaborn as sns
import numpy as np
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import itertools
import collections
from nltk import bigrams
import networkx as nx
import re
import nltk
import plotly
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [ ]:
app = Flask(__name__)
app.config['IMAGES_PATH'] = os.path.join('static', 'images')

In [ ]:
def clean_tweet(tweet):
    #Step 1 - drop links
    regex = re.compile(r'https?://t.co/[a-zA-Z0-9]{10}')
    remove_links = re.split(regex,tweet)
    remove_links = ' '.join(remove_links)
    
    #Step 2 - Remove any non-ascii characters
    remove_non_ascii =  re.sub(r'[^\x00-\x7F]+',' ', remove_links).lower()

    #Step 3 - check for apostrophes
    remove_apostroph = remove_non_ascii.replace("'s","")
    remove_apostroph = remove_apostroph.replace("'","")
    remove_apostroph = remove_apostroph.split(' ')
    
    #Step 4 - Remove dashes
    remove_dashes = ' '.join(remove_apostroph).split('-')
    remove_dashes = ' '.join(remove_dashes)

    #Step 5 - Keep letters and digits only
    keep_letters =  re.sub(r'[^a-zA-Z0-9]',' ', remove_dashes)

    keep_letters = keep_letters.split(' ')

    keep_letters = ' '.join(keep_letters)

    #Step 6 - Tokenize 
    tokenize = nltk.word_tokenize(keep_letters)

    #Step 7 - Lemmatize    
    lemmatizer= nltk.stem.wordnet.WordNetLemmatizer()
    lemmatize_text = map(lambda x: lemmatizer.lemmatize(x), tokenize)

    #Step 8 - Remove stop words
    stopwords = set(nltk.corpus.stopwords.words('english'))
    remove_stop = list(filter(lambda x: x not in stopwords, lemmatize_text))

    #Step 9 - Remove any empty strings from list
    processed_list = list(filter(lambda a: a != "", remove_stop))

    processed_string = " ".join(processed_list)

    return processed_string

def sent_to_list(sentence):
    return sentence.split(" ")

def sentiment_analyze(polarity):
    if (polarity < 0.1 and polarity > -0.1): return 'neutral'
    elif polarity >= 0.1: return 'positive'
    else: return 'negative'

In [ ]:
df = pd.read_csv('dataset/final_tweets.csv')
df['Year'] = df['Date'].apply(lambda r: r.split('/')[2])
df = df[df['Year'] == '2021']
df.drop(columns=['Year'], inplace=True)
df['Date'] = df['Date'].apply(lambda r: r.split('/')[0])
df['Tweet'] = df['Tweet'].apply(lambda tweet: clean_tweet(tweet))
df['Words_list'] = df['Tweet'].apply(lambda tweet: sent_to_list(tweet))

sentiment_objs = [TextBlob(tweet) for tweet in df['Tweet']]
all_senti =[tweet.sentiment.polarity for tweet in sentiment_objs]
sentiment_vals = [[tweet.sentiment.polarity, str(tweet)] for tweet in sentiment_objs]
sentiment_df = pd.DataFrame(sentiment_vals, columns=["polarity", "tweet"])

neg = sentiment_df['polarity'].astype('float')
neg = neg[neg <= -0.1]

pos = sentiment_df['polarity'].astype('float')
pos = pos[pos >= 0.1]

neut = sentiment_df['polarity'].astype('float')
neut = neut[neut > -0.1]
neut = neut[neut < 0.1]
df['Sentiment'] = sentiment_df['polarity']
df['Sentiment'] = df['Sentiment'].apply(lambda x: sentiment_analyze(x))

df1 = df
polarity_by_month = df1[['Date','Tweet','Sentiment']].groupby(['Date','Sentiment'])['Tweet'].count().reset_index().rename(columns={'Tweet':'Tweet_Count'})

keywords_by_date = df[['Keyword', 'Date']]
keywords_by_date = keywords_by_date[['Keyword', 'Date']].groupby(['Date', 'Keyword'])['Keyword'].count()

count = 0
month = 0
final_vals = []
for i in range(0,len(keywords_by_date)):
  keyword = "Covid" if count%2==0 else "Corona"
  month = month + 1 if count%2==0 else month
  count = count + 1
  counts = keywords_by_date[i]
  current = [month, keyword, counts]
  final_vals.append(current)

df2 = pd.DataFrame(final_vals, columns=['Month', 'Keyword', 'Keyword_Count'])
w2v_model = Word2Vec()
w2v_model.build_vocab(df['Words_list'].tolist())
w2v_model.train(df['Words_list'].tolist(), total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

bigram_terms = [list(bigrams(tweet)) for tweet in df['Words_list'].tolist()]

In [ ]:
tweet_polarity = px.bar(df,
                          x=['Positive', 'Neutral', 'Negative'],
                          y=[len(pos), len(neut), len(neg)],
                          labels = {'x':'Distribution of Sentiments', 'y':'Number of Tweets'}
                        )
tw_polarity = px.line(polarity_by_month, x='Date', y='Tweet_Count', color='Sentiment', color_discrete_sequence=['brown','orange','green'])
f = px.line(df2, x='Month', y='Keyword_Count', color='Keyword', color_discrete_sequence=['brown', 'magenta'])

In [ ]:
sentiments = {'positive': 0,'negative': 0, 'total': 0}
hashtag_counts = {'words': ['#SocialDistancing'], 'counts': [16]}
word_counts = {'words': ['COVID19', 'Quarantine'], 'counts': [16, 50]}

In [ ]:
@app.route("/", methods=['GET', 'POST'])
def home_page():
    graph1JSON = json.dumps(tweet_polarity, cls=plotly.utils.PlotlyJSONEncoder)
    graph2JSON = json.dumps(tw_polarity, cls=plotly.utils.PlotlyJSONEncoder)
    graph3JSON = json.dumps(f, cls=plotly.utils.PlotlyJSONEncoder)
    return render_template('blockcontent.html',
                           graph1JSON = graph1JSON, 
                           graph2JSON = graph2JSON, 
                           graph3JSON = graph3JSON)

In [ ]:
@app.route("/real-time")
def realTime_page():
    global sentiments
    global hashtag_counts
    global word_counts
    global geodata
    global graphJSON

    print("word count variable", file=sys.stderr)
    print(word_counts, file=sys.stderr)
    print("hashtag variable", file=sys.stderr)
    print(hashtag_counts, file=sys.stderr)
    print(hashtag_counts['words'], file=sys.stderr)
    
    wc = dict(zip(hashtag_counts['words'], hashtag_counts['counts']))


    return render_template(
            'index.html',
            sentiments=sentiments,
            wordcounts=word_counts,
            hashtagcounts=hashtag_counts)

In [ ]:
@app.route('/real-time/update_sentiments', methods=['POST'])
def update_sentiments():
    global sentiments

    print(request.form)

    sentiments['positive'] = ast.literal_eval(request.form['positive'])
    sentiments['neutral'] = ast.literal_eval(request.form['neutral'])
    sentiments['negative'] = ast.literal_eval(request.form['negative'])
    sentiments['total'] = ast.literal_eval(request.form['total'])

    if sentiments['total'] > 0:
        sentiments['positive'] = round(100 * sentiments['positive'] / sentiments['total'], 2)
        sentiments['neutral'] = round(100 * sentiments['neutral'] / sentiments['total'], 2)
        sentiments['negative'] = round(100 * sentiments['negative'] / sentiments['total'], 2)

    return "success", 200

In [ ]:
@app.route('/real-time/update_counts', methods=['POST'])
def update_counts():
    global word_counts
    
    print(request.form)
    
    word_counts['words'] = ast.literal_eval(request.form['words'])
    word_counts['counts'] = ast.literal_eval(request.form['counts'])
    print("Updated word counts - ",  word_counts)
    return "success", 200

In [ ]:
@app.route('/real-time/update_hashtagcounts', methods=['POST'])
def update_hashtagcounts():
    global hashtag_counts
    
    print(request.form)
    
    hashtag_counts['words'] = ast.literal_eval(request.form['words'])
    hashtag_counts['counts'] = ast.literal_eval(request.form['counts'])
    print("Updated hashtag counts - ",  hashtag_counts)
    return "success", 200

In [ ]:
@app.route('/real-time/word_counts', methods=['GET'])
def refresh_counts():
    global word_counts
    output = json.dumps(word_counts)
    print(output)
    return output

In [ ]:
@app.route('/real-time/hashtag_counts', methods=['GET'])
def refresh_hashtagcounts():
    global hashtag_counts
    output = json.dumps(hashtag_counts)
    print(output)
    return output

In [ ]:
@app.route('/real-time/sentiments', methods=['GET'])
def refresh_sentiments():
    global sentiments
    print(sentiments)
    return sentiments

In [ ]:
@app.after_request
def add_header(response):
    response.headers['X-UA-Compatible'] = 'IE=Edge,chrome=1'
    response.headers['Cache-Control'] = 'public, max-age=0'
    return response

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)